In [1]:
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook
tqdm.pandas()

import pandas as pd

In [3]:
from utils import filter_by_media
from utils import cluster_by_month
#from utils import preprocess

df = pd.read_csv("../../data/loslagos-comunas.csv")
df = cluster_by_month(filter_by_media(df))
df = df.drop_duplicates(subset='content', keep="first")

## 1. Modelado de tópicos con BERTopic

In [4]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
import spacy
from umap import UMAP
from hdbscan import HDBSCAN

nlp = spacy.load("es_core_news_md", exclude=['tagger', 'parser', 'ner', 
                                             'attribute_ruler', 'lemmatizer'])

In [5]:
df['topic'] = ""
docs = df.content.tolist()
df.head()

,date,media_outlet,url,title,text,content,comuna,date_clustering,topic
0,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/reconocen-a-g...,Reconocen a guardaparques de la Región de Los ...,Distintos protagonistas de los parques naciona...,reconocen guardaparques región lagos actores c...,"['puyehue', 'chaiten']",2021-10,
1,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/con-nuevos-ma...,Con nuevos materiales comienza plan piloto en ...,Centro de negocios Sercotec coordina acuerdos ...,nuevos materiales comienza plan piloto saltos ...,['puerto varas'],2021-10,
2,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/centro-de-sal...,Centro de Salud Familiar CESFAM Puerto Varas i...,Las horas se solicitan en el SOME o bien a tra...,centro salud familiar cesfam puerto varas invi...,['puerto varas'],2021-10,
3,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/alcalde-tomas...,Alcalde Tomás Gárate presidió por primera vez ...,Los y las consejeras destacaron el hecho de vo...,alcalde tomás gárate presidió primera vez octa...,"['castro', 'puerto varas']",2021-10,
4,2021-10-01,elheraldoaustral,https://www.eha.cl/noticia/local/galeria-de-ar...,Galería de Arte Machacoya realizará remate de ...,"Hoy viernes a las 18:30 horas, en Machacoya At...",galería arte machacoya realizará remate obras ...,NaN,2021-10,


In [ ]:
umap_model = UMAP(n_neighbors=15, n_components=5, 
                  min_dist=0.0, metric='cosine', random_state=42)

hdbscan_model = HDBSCAN(min_cluster_size=10, metric='euclidean', 
                        cluster_selection_method='eom', prediction_data=True, min_samples=5)

In [9]:
vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words=None)

topic_model = BERTopic(#language="multilingual",
                       embedding_model=nlp,
                       vectorizer_model=vectorizer_model,
                       umap_model=umap_model,
                       hdbscan_model=hdbscan_model,
                       calculate_probabilities=True,
                       verbose=True,
                       diversity=0.6)

topics, probs = topic_model.fit_transform(docs)

100%|███████████████████████████████████████████████████████████████████████████| 26127/26127 [03:38<00:00, 119.41it/s]
2022-11-02 01:31:27,559 - BERTopic - Transformed documents to Embeddings
2022-11-02 01:31:52,175 - BERTopic - Reduced dimensionality
2022-11-02 01:36:58,828 - BERTopic - Clustered reduced embeddings


In [10]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,10553,-1_parte_trabajo_ser_comuna
1,0,1021,0_colo_partido_selección_jugar
2,1,528,1_guerra_otan_rusos_fuerzas
3,2,482,2_minutos_betis_tras_final
4,3,408,3_boric_kast_candidato_antonio kast
...,...,...,...
403,402,10,402_turismo_cómo_se_sino
404,403,10,403_deportes puerto_brayan_ciampichetti_arnaldo
405,404,10,404_lote_wwwsubastaaduaneracl_subastas_mínimo
406,405,10,405_chiguayante_garc茅s_enap_garcés


In [8]:
topic_model.visualize_topics()

MemoryError: Unable to allocate 22.0 GiB for an array with shape (408, 7241953) and data type float64